In [ ]:
# -*- coding: utf-8 -*-
import os
import json
import time
import random
import re
import torch
import numpy as np
# from transformers import BertTokenizer, BertModel # Not used directly
from tqdm import tqdm
from dataclasses import dataclass, asdict
from typing import List, Dict, Any, Optional
import logging
import requests
# from sklearn.metrics.pairwise import cosine_similarity # Not used directly
import gc

# 设置日志
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("essay_comment_generation.log", encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# 设置随机种子
def set_seed(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed()

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info(f"使用设备: {device}")

# 设置API相关参数
API_URL = "url"
API_MODEL = "qwen-max"
API_MAX_TOKENS = 8192
API_TEMPERATURE = 0.1
API_STREAM = False

# API密钥列表 (需要替换为您的密钥)
API_KEYS = [
    "apikeys",
    "apikeys",
    "apikeys",
    "apikeys",
    "apikeys"
]

# 数据路径
data_root = "."
input_file = os.path.join(data_root, "test_data.json")
output_file = os.path.join(data_root, "DUFL2025_track2.json")
results_file = os.path.join(data_root, "DUFL2025_track1.json")
samples_file = os.path.join(data_root, "samples.json")

# 缓存路径
cache_dir = os.path.join(data_root, ".cache")
processed_comments_file = os.path.join(data_root, "processed_comments_cache_track2.json")

if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)

@dataclass
class Message:
    content: str
    role: str

@dataclass
class ApiRequest:
    messages: List[dict]
    model: str
    max_tokens: Optional[int] = None
    temperature: Optional[float] = None
    stream: Optional[bool] = None

# --- API 密钥管理 & 调用 ---
current_api_idx = 0
last_switch_time = time.time()

def switch_api_key():
    global current_api_idx, last_switch_time
    if not API_KEYS:
        logger.error("API_KEYS 列表为空，无法切换。")
        return None
    current_api_idx = (current_api_idx + 1) % len(API_KEYS)
    last_switch_time = time.time()
    logger.info(f"切换到 API 密钥索引 {current_api_idx}")
    return API_KEYS[current_api_idx]

def get_current_api_key():
    if not API_KEYS:
        raise ValueError("API_KEYS 列表为空。请提供有效的 API 密钥。")
    return API_KEYS[current_api_idx]

def call_llm_api(messages: List[Message], retry_on_empty=True, agent_name="Unknown Agent"):
    """使用 requests 调用 LLM API (同步), 集成重试和密钥轮换"""
    url = f"https://{API_URL}/v1/chat/completions"
    messages_dict_list = [asdict(m) for m in messages]
    # Adjust max_tokens based on desired output length (heuristic)
    # Assuming ~2 chars per token for Chinese, aiming for ~200 chars needs ~100 tokens.
    # Add buffer. Use a higher value like 512 or 1024 to be safe, as we have truncation logic.
    # Let's keep API_MAX_TOKENS as is for now, as it's an upper limit.
    # The prompt should guide the length more effectively.
    model_request = ApiRequest(model=API_MODEL, messages=messages_dict_list, max_tokens=API_MAX_TOKENS, temperature=API_TEMPERATURE, stream=API_STREAM)
    payload_dict = {k: v for k, v in asdict(model_request).items() if v is not None}
    payload = json.dumps(payload_dict)
    num_retries = 5

    for attempt in range(num_retries):
        current_key = get_current_api_key()
        if not current_key:
            logger.error(f"[{agent_name}] 无可用 API 密钥。")
            return f"API_ERROR: [{agent_name}] 无可用 API 密钥"
        headers = {"Authorization": f"Bearer {current_key}", "Content-Type": "application/json"}
        try:
            time.sleep(random.uniform(0.8, 1.8))
            logger.debug(f"[{agent_name}] 尝试 {attempt + 1}/{num_retries}: 调用 API")
            response = requests.post(url, data=payload, headers=headers, timeout=240)
            logger.debug(f"[{agent_name}] 尝试 {attempt + 1}: 收到状态码 {response.status_code}")

            if response.status_code == 429:
                logger.warning(f"[{agent_name}] 速率限制 (429)。尝试 {attempt + 1}/{num_retries}。切换密钥并延迟重试。")
                new_key = switch_api_key()
                if not new_key:
                    return f"API_ERROR: [{agent_name}] 切换后无可用 API 密钥"
                wait_time = (1.5 ** (attempt + 1)) + random.uniform(0, 1)
                logger.info(f"[{agent_name}] 速率限制，将在 {wait_time:.2f} 秒后重试...")
                time.sleep(wait_time)
                continue

            response.raise_for_status()
            json_data = response.json()
            logger.debug(f"[{agent_name}] 尝试 {attempt + 1}: 收到 JSON。")

            if ("choices" in json_data and len(json_data["choices"]) > 0 and
                    "message" in json_data["choices"][0] and "content" in json_data["choices"][0]["message"]):
                content = json_data["choices"][0]["message"]["content"]
                if retry_on_empty and (not content or content.strip() == ""):
                    logger.warning(f"[{agent_name}] 空响应 (尝试 {attempt + 1}/{num_retries})")
                    if attempt < num_retries - 1:
                        time.sleep(1.5**(attempt + 1))
                        continue
                    else:
                        logger.error(f"[{agent_name}] 最大重试后仍空响应。")
                        return f"API_ERROR: [{agent_name}] 最大重试后响应为空"
                logger.debug(f"[{agent_name}] 尝试 {attempt + 1}: 成功。")
                return content.strip()
            else:
                logger.error(f"[{agent_name}] 无效响应结构: {json_data} (尝试 {attempt + 1}/{num_retries})")
                return f"API_ERROR: [{agent_name}] 无效响应结构"
        except requests.exceptions.HTTPError as e:
            logger.error(f"[{agent_name}] HTTP 错误 (尝试 {attempt + 1}/{num_retries}): {e.response.status_code}, URL: {e.request.url}")
            try:
                logger.error(f"[{agent_name}] 错误响应体: {e.response.text[:500]}...")
            except Exception as body_e:
                logger.error(f"[{agent_name}] 无法读错误响应体: {body_e}")
            if e.response.status_code == 401:
                logger.error(f"[{agent_name}] 授权错误 (401)。切换密钥。")
                new_key = switch_api_key()
                if not new_key:
                    return f"API_ERROR: [{agent_name}] 401 后切换密钥失败"
            elif e.response.status_code == 400:
                logger.error(f"[{agent_name}] 错误请求 (400)。Payload: {payload[:200]}...")
                return f"API_ERROR: [{agent_name}] 错误请求 (400)"
        except (requests.exceptions.ConnectionError, requests.exceptions.Timeout) as e:
            logger.error(f"[{agent_name}] 网络/连接错误 (尝试 {attempt + 1}/{num_retries}): {type(e).__name__} - {e}")
        except json.JSONDecodeError as e:
            logger.error(f"[{agent_name}] JSON 解码错误 (尝试 {attempt + 1}/{num_retries}): {e}")
            try:
                logger.error(f"[{agent_name}] 原始响应文本: {response.text[:500]}...")
            except Exception as text_e:
                logger.error(f"[{agent_name}] 无法获取原始响应文本: {text_e}")
        except Exception as e:
            logger.error(f"[{agent_name}] API 调用意外错误 (尝试 {attempt + 1}/{num_retries}): {type(e).__name__} - {e}", exc_info=True)

        if attempt < num_retries - 1:
            wait_time = (1.5 ** (attempt + 1)) + random.uniform(0, 1)
            logger.info(f"[{agent_name}] 将在 {wait_time:.2f} 秒后重试...")
            time.sleep(wait_time)
        else:
            logger.error(f"[{agent_name}] API 调用达到最大重试次数 {num_retries}。")
            return f"API_ERROR: [{agent_name}] 达到最大重试次数"
    logger.error(f"[{agent_name}] API 调用循环完成但未返回。")
    return f"API_ERROR: [{agent_name}] 未知失败"


# --- 评论清理函数 ---
def clean_comment(comment):
    if not isinstance(comment, str): return ""
    comment = comment.strip()
    comment = comment.replace('\n', ' ').replace('\r', ' ')
    comment = re.sub(r'(\*\*|__|\*|_|~~|`)', '', comment)
    comment = re.sub(r'<[^>]+>', '', comment)
    comment = re.sub(r'^\s*(评语|点评|最终评语|最终的评语|comment|feedback|final comment)[:：\s]*', '', comment, flags=re.IGNORECASE | re.UNICODE)
    comment = re.sub(r'^\s*[\d]+\.\s*|^\s*-\s*', '', comment).strip()
    comment = re.sub(r'[「」『』]', '', comment)
    # Keep length related cleaning for now, might help LLM output
    comment = re.sub(r'\s*（\s*\d+\s*[字字符]?\s*）\s*$', '', comment)
    comment = re.sub(r'\s*\(\s*\d+\s*chars?\s*\)\s*$', '', comment, flags=re.IGNORECASE)
    comment = re.sub(r'\[\s*\d+\s*字\s*\]\s*$', '', comment)
    comment = re.sub(r'\s{2,}', ' ', comment).strip()
    comment = re.sub(r'^\s*最终评语\s*[:：]?\s*', '', comment, flags=re.IGNORECASE)
    comment = re.sub(r'^\s*润色后的评语\s*[:：]?\s*', '', comment, flags=re.IGNORECASE)
    comment = re.sub(r'^\s*合规评语\s*[:：]?\s*', '', comment, flags=re.IGNORECASE)
    return comment

# --- 缓存管理 ---
def load_processed_comments():
    if os.path.exists(processed_comments_file):
        try:
            with open(processed_comments_file, 'r', encoding='utf-8') as f:
                content = f.read()
            if not content:
                logger.warning(f"缓存文件 '{processed_comments_file}' 为空。")
                return {}
            data = json.loads(content)
            if isinstance(data, dict):
                logger.info(f"从 '{processed_comments_file}' 加载 {len(data)} 条缓存。")
                return data
            else:
                logger.error(f"缓存文件 '{processed_comments_file}' 格式错误。")
                return {}
        except json.JSONDecodeError as e:
            logger.error(f"解码缓存文件 '{processed_comments_file}' 出错: {e}。")
            try:
                backup_path = processed_comments_file + f".corrupted_{int(time.time())}"
                os.rename(processed_comments_file, backup_path)
                logger.info(f"备份损坏缓存到 {backup_path}")
            except OSError as rename_e:
                logger.error(f"无法备份损坏缓存: {rename_e}")
            return {}
        except Exception as e:
            logger.error(f"加载缓存文件 '{processed_comments_file}' 出错: {e}。", exc_info=True)
            return {}
    else:
        logger.info(f"缓存文件 '{processed_comments_file}' 未找到。")
        return {}

def save_processed_comments(processed_comments):
    try:
        temp_file_path = processed_comments_file + ".tmp"
        with open(temp_file_path, 'w', encoding='utf-8') as f:
            json.dump(processed_comments, f, ensure_ascii=False, indent=2)
        os.replace(temp_file_path, processed_comments_file)
        logger.debug(f"保存 {len(processed_comments)} 条缓存到 '{processed_comments_file}'")
    except Exception as e:
        logger.error(f"保存缓存到 '{processed_comments_file}' 出错: {e}", exc_info=True)
        if os.path.exists(temp_file_path):
            try:
                os.remove(temp_file_path)
            except OSError as remove_error:
                logger.error(f"移除临时缓存文件 {temp_file_path} 出错: {remove_error}")

# --- 加载样本数据 ---
def load_samples(file_path):
    if not os.path.exists(file_path):
        logger.warning(f"样本文件未找到: {file_path}。")
        return []
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            samples = json.load(f)
        if not isinstance(samples, list):
            logger.error(f"样本文件顶层不是列表: {file_path}。")
            return []
        valid_samples = []
        required_keys = {"grade", "requirement", "title", "content", "classification", "comment"}
        valid_classifications = {"优秀", "较好", "一般", "合格", "不合格"}
        for i, sample in enumerate(samples):
            if isinstance(sample, dict) and required_keys.issubset(sample.keys()):
                if sample.get("classification") in valid_classifications:
                    valid_samples.append(sample)
                else:
                    logger.warning(f"样本 {i+1} classification 无效 ('{sample.get('classification')}')，已跳过。")
            else:
                logger.warning(f"样本 {i+1} 格式无效或缺键，已跳过。样本: {str(sample)[:100]}...")
        logger.info(f"成功加载 {len(valid_samples)} 条有效样本从 {file_path}。")
        return valid_samples
    except json.JSONDecodeError as e:
        logger.error(f"解析样本文件失败: {file_path} - {e}。")
        return []
    except Exception as e:
        logger.error(f"加载样本数据出错: {e}", exc_info=True)
        return []

# --- 格式化样本 (按分类筛选) ---
def format_samples_for_prompt(samples_data, target_classification, max_samples=3):
    if not samples_data or not target_classification:
        return ""
    matching_samples = [s for s in samples_data if s.get('classification') == target_classification]
    if not matching_samples:
        logger.debug(f"未找到分类为 '{target_classification}' 的样本。")
        return ""
    formatted_string = f"\n为帮助你更好地完成任务，可以参考以下同类（{target_classification}）的高质量范例。学习它们的分析逻辑、评论结构和措辞。\n"
    samples_to_use = matching_samples[:max_samples]
    for i, sample in enumerate(samples_to_use):
        requirement_full = sample.get('requirement', 'N/A')
        formatted_string += (
            f"\n--- 同类范例 {i+1} ({sample.get('classification', 'N/A')}) ---\n"
            f"年级: {sample.get('grade', 'N/A')}\n"
            f"要求: {requirement_full}\n"
            f"标题: {sample.get('title', 'N/A')}\n"
            f"参考评论 (仅关注切题度与中心思想): {sample.get('comment', 'N/A')}\n"
        )
    formatted_string += "\n以上为全部学习范例。\n"
    return formatted_string

# --- 加载 Track 1 结果 ---
def load_classification_results(results_file_path):
    if not os.path.exists(results_file_path):
        logger.error(f"分类结果文件未找到: {results_file_path}。")
        return {}
    try:
        with open(results_file_path, 'r', encoding='utf-8') as f:
            results_data = json.load(f)
        if not isinstance(results_data, list):
            logger.error(f"分类结果文件 '{results_file_path}' 顶层不是列表。")
            return {}
        results_dict = {}
        valid_classifications = {"优秀", "较好", "一般", "合格", "不合格"}
        for item in results_data:
            item_id = item.get("id")
            classification = item.get("classification")
            if item_id is not None and isinstance(item_id, int) and classification is not None:
                if classification in valid_classifications:
                    results_dict[str(item_id)] = classification
                else:
                    logger.warning(f"跳过无效分类 '{classification}' (ID: {item_id})。使用默认 '一般'。")
                    results_dict[str(item_id)] = "一般"
            else:
                logger.warning(f"跳过结果文件无效条目: {item}")
        logger.info(f"成功加载 {len(results_dict)} 条有效分类结果。")
        return results_dict
    except json.JSONDecodeError:
        logger.error(f"解析分类结果文件失败: {results_file_path}")
        return {}
    except Exception as e:
        logger.error(f"加载分类结果出错: {e}", exc_info=True)
        return {}

# --- Agent 定义 (保持不变) ---

# Agent 0: 作文类型识别智能体
def generate_essay_type_classifier_prompt(essay_data: Dict[str, Any]) -> List[Message]:
    grade = essay_data.get("grade", "指定")
    requirement = essay_data.get("requirement", "")
    title = essay_data.get("title", "")
    content = essay_data.get("content", "")
    system_prompt = f"""角色: 中小学作文类型识别专家，专注于分析作文的体裁类型，具备深厚的文体理论知识和丰富的教学经验。

核心任务: 对照《义务教育语文课程标准》对应学段写作要求，精准分析提供的{grade}年级学生作文，识别其最主要的文体归属（例如：记叙文、议论文、说明文、应用文、读后感/观后感、想象文、散文、诗歌、戏剧、小说等）。**你的判断对后续的切题分析和评语生成至关重要**。

分析维度:
1. **题目要求解读**: 从写作要求（requirement）中寻找明确的文体指令或强烈暗示，这是判断学生是否理解题目的首要依据。
2. **内容与结构**: 分析文章的核心内容（写人、记事、状物、说理、说明、抒情等）及篇章结构（时间顺序、空间顺序、逻辑顺序、总分总等）是否符合特定文体的典型特征和题目要求。
3. **表达方式**: 考察主要的表达方式（记叙、描写、抒情、说明、议论）及其在文中的运用比例和目的，判断是否与题目要求的文体相符。
4. **语言风格**: 判断语言是偏向形象生动、平实准确还是逻辑严谨，评估是否适合题目要求的表达需求。
5. **标题指向**: 标题（title）是否能提示文体类型且与题目要求保持一致？
6. **年级特征**: 结合{grade}年级学生的写作能力和常见训练重点进行综合判断，考虑该年级学生对文体的理解深度。

输出要求:
生成一份简明扼要的《作文类型识别报告》，必须包含且仅包含以下两项：
一、**主要文体类型**: 从常见文体中明确选择一个最贴切的类型名称（例如："记叙文" 或 "议论文"）。**不允许出现模棱两可或多种类型并列**。请使用规范的文体术语。
二、**核心判断依据**: 列出 2-3 条最关键、最有说服力的证据（可引用原文片段或概括特征），直接支撑第一项的文体判断。**特别注意判断该文体是否符合题目明确或隐含要求**。

请务必做到：判断精准、依据充分、表达清晰。**避免在报告中提及任何与主要文体判断和题目相关性无关的评论**。你的分析将作为后续评估学生作文切题性的重要基础。
"""
    user_prompt = f"""请依据上述要求，分析以下这篇{grade}年级学生作文：

**题目要求 (Requirement):**
{requirement}

**作文标题 (Title):**
{title}

**作文内容 (Content):**
{content}

请输出《作文类型识别报告》，明确指出其最主要的文体类型，并提供2-3条核心判断依据。**特别注意评估该文体选择是否符合题目要求，这将直接影响后续的切题性分析。**"""
    return [Message(role="system", content=system_prompt), Message(role="user", content=user_prompt)]

def extract_essay_type(response: str) -> str:
    """从 Agent 0 的响应中提取主要文体类型"""
    if not response or 'API_ERROR' in response:
        logger.warning("Agent 0 响应无效，默认返回 '记叙文'")
        return "记叙文"
    match = re.search(r"(主要文体类型|文体类型|体裁类型|判断为|类型是)\s*[:：]?\s*([^\s，。；（(]+(文|记|论|说明|应用|诗歌|戏剧|小说|散文|故事|书信))", response)
    if match:
        extracted_type = match.group(2).strip()
        logger.debug(f"标签提取类型: {extracted_type}")
        # 标准化常见类型
        if "记叙" in extracted_type: return "记叙文"
        if "议论" in extracted_type: return "议论文"
        if "说明" in extracted_type: return "说明文"
        if "应用" in extracted_type: return "应用文"
        if "读后感" in extracted_type or "观后感" in extracted_type: return "读后感/观后感"
        if "想象" in extracted_type or "科幻" in extracted_type: return "想象文"
        if "散文" in extracted_type: return "散文"
        if "诗歌" in extracted_type: return "诗歌"
        if "戏剧" in extracted_type: return "戏剧"
        if "小说" in extracted_type: return "小说"
        if "童话" in extracted_type: return "童话故事"
        if "推荐" in extracted_type: return "推荐信/推荐文"
        if "游记" in extracted_type: return "游记"
        return extracted_type
    types_ordered = ["议论文", "说明文", "应用文", "记叙文", "读后感", "观后感", "想象文", "推荐信", "推荐文", "游记", "童话故事", "散文", "诗歌", "戏剧", "小说"]
    for essay_type in types_ordered:
        if re.search(r'\b' + re.escape(essay_type) + r'\b', response):
            logger.debug(f"关键词提取类型: {essay_type}")
            return essay_type
    match_suffix = re.search(r"([^\s，。；（(]{2,})(文|记|论|说明|应用|诗歌|戏剧|小说|散文|故事|书信)\b", response)
    if match_suffix:
         extracted_suffix_type = match_suffix.group(1) + match_suffix.group(2)
         logger.warning(f"后缀提取类型: {extracted_suffix_type}")
         return extracted_suffix_type
    logger.warning(f"无法提取作文类型，默认 '记叙文'。响应: {response[:200]}...")
    return "记叙文"

# Agent 1: 题目要求解析器
def generate_prompt_analyzer_prompt(essay_data: Dict[str, Any], essay_type: Optional[str]) -> List[Message]:
    grade = essay_data.get("grade", "指定")
    requirement = essay_data.get("requirement", "")
    type_info = f"背景信息：预设文体类型倾向于“{essay_type}”。" if essay_type else ""
    system_prompt = f"""你是一位资深的中文中小学作文题目解构专家，兼任题目解析专家，精通各类文体（特别是{essay_type or '常见文体'}）的命题特点与考察意图。
    任务: 对以下{grade}年级的作文题目要求进行**深度、客观、精细化**的分析，提炼所有关键信息点，为后续的切题度评估奠定坚实基础。
{type_info}
你需要关注并列出：

分析框架（必须包含以下要素，如题目未明确涉及则注明"未明确"）：
1.  核心写作任务: 明确要求学生"做什么"？（例如：记叙、描写、议论、说明等具体任务）
2.  主题/内容范围: 明确要求学生"写什么"？（具体事件、人物、情感或观点等）
3.  文体要求: 题目是否明确规定或强烈暗示了文体？（记叙文、议论文、说明文等）
4.  关键要素/约束: 是否有必须包含的人物、情节、细节、关键词、角度、情感基调、人称等硬性规定？
5.  隐含考察点: 结合{grade}年级特点和{essay_type or '文体'}特性，推断题目可能侧重考察学生哪些能力？（如观察力、思辨能力、表达能力等）

输出要求:
生成一份结构清晰、逻辑严谨的《题目要求分析报告》。语言务必**客观中立**，仅呈现对题目本身的解读，**严禁**加入任何对题目优劣的评价、对学生写作的建议或主观臆断。确保分析全面、精准、无遗漏。"""

    user_prompt = f"""请按照上述分析框架，深度解析以下{grade}年级作文题目要求：
{requirement}

请输出结构化的《题目要求分析报告》。"""

    user_prompt = f"""请按照上述分析框架，深度解析以下{grade}年级作文题目要求：

**题目要求 (Requirement):**
{requirement}

请输出结构化的《题目要求分析报告》。"""
    return [Message(role="system", content=system_prompt), Message(role="user", content=user_prompt)]

# Agent 2: 作文主旨提取器
def generate_essay_theme_extractor_prompt(essay_data: Dict[str, Any], essay_type: Optional[str]) -> List[Message]:
    grade = essay_data.get("grade", "指定")
    title = essay_data.get("title", "")
    content = essay_data.get("content", "")
    type_info = f"背景信息：预设文体类型倾向于“{essay_type}”。" if essay_type else ""
    system_prompt = f"""你是一位专业的中文中小学作文文本主旨分析师，具备强大的文本理解和信息概括能力。
    任务: **客观、准确、完整**地分析并提炼出以下{grade}年级学生作文实际表达的核心内容和中心思想（或主旨大意）。
{type_info}
分析要点:
1. **核心叙述/说明/论证内容**: 文章主体部分具体写了什么人物、事件、景物、过程、观点或道理？请用简洁的语言概括，**确保涵盖作文的关键要素**。
2. **中心思想/主旨**: 作者通过上述内容，最终想要传达的核心信息、观点、情感、感悟或揭示的道理是什么？**注重挖掘文本深层含义，不添加个人解读**。
3. **标题与主旨关联**: 标题在多大程度上概括或暗示了文章的中心思想？**分析标题与内容的内在联系**。

输出要求:
生成一份简洁、客观的《作文主旨分析报告》，包含对上述三个要点的分析。
**极端重要**: 你的任务是**"如实报告"**文章写了什么和表达了什么，**绝对禁止**包含任何关于作文质量、写作技巧、是否切题、错别字等方面的评价或暗示。只聚焦于**内容本身及其传递的核心信息**。报告语言需中性、精确，**避免主观色彩**。"""
    user_prompt = f"""请根据以上要求，客观分析以下{grade}年级学生作文，提取其核心内容与中心思想：

**作文标题 (Title):**
{title}

**作文内容 (Content):**
{content}

请输出客观、不含评价的《作文主旨分析报告》。"""
    return [Message(role="system", content=system_prompt), Message(role="user", content=user_prompt)]

# --- 新 Agent A: 年级适配性评估器 ---
def generate_grade_adaptability_assessor_prompt_A(
    essay_data: Dict[str, Any],
    essay_type: Optional[str],
    predicted_classification: str # 使用预测分类作为主要输入
) -> List[Message]:
    """为新 Agent A (年级适配性评估器) 生成 Prompt"""
    grade = essay_data.get("grade", "指定")
    requirement = essay_data.get("requirement", "") # 用于参考

    system_prompt = f"""角色: 经验丰富的中小学语文教研员，熟悉各年级语文课程标准和学生认知发展规律。
任务: 基于作文的基本信息（年级、题目要求）、识别出的文体类型以及**预设的整体分类（{predicted_classification}）**，评估作文在**切题方面的表现是否符合其所在年级（{grade}年级）学生的普遍认知水平和写作能力要求**。

评估维度与输出结构 (请严格按此格式输出《年级适配性评估报告》):

1.  **审题能力适配性 (Topic Comprehension Adaptability):**
    *   结合预设分类 ({predicted_classification}) 深入分析该生对题目核心要求的理解深度和准确性。
    *   评估这种理解深度是否符合{grade}年级学生应有的审题能力？（明确区分：显著超出预期、符合预期、略低于预期、远低于预期）
    *   提供具体依据说明理由（例如："分类为'优秀'表明其能精准把握题目隐含要求并融入创新视角，超出该年级典型水平"；或"分类为'合格'表明仅理解表层含义而忽略深层内涵，低于该年级应有审题深度"）。
2.  **内容回应适配性 (Content Response Adaptability):**
    *   基于预设分类 ({predicted_classification})，分析作文在围绕中心主题选取素材、组织内容以及表达思想方面的切题表现。
    *   精确评估这种表现与{grade}年级学生课程标准规定的能力水平的契合度（例如："分类'较好'表明选材具针对性且层次分明，符合{grade}年级学生预期水平"；或"分类'不合格'表明内容散乱无序且偏离核心要求，未达{grade}年级基本能力要求"）。
    *   结合文体类型（{essay_type or '未知'}），评价其在运用该文体元素和特征回应题目要求方面是否达到{grade}年级教学目标的具体表现。
3.  **切题表现综合评价 (Overall Relevance Adaptability):**
    *   综合前两项分析，客观评价该作文的**切题表现**（仅限切题度）与{grade}年级语文课程标准规定的学生能力指标相比达到何种水平？（精准对应 {predicted_classification} 的等级描述，如：优秀水平、良好水平、中等水平、有待提高、严重不足）
    *   清晰说明此评价是基于{grade}年级教学大纲和能力要求的专业判断，具体指出作文在切题方面的优势或不足。

请确保评估紧密结合**{grade}年级**的语文教学大纲、年龄认知特点和写作能力目标，使用学科专业术语进行分析，始终聚焦于"切题度适配性"而非作文的其他方面。**严格避免评论语言表达、文采、字迹等与切题无关的因素**。你的分析将为后续评语提供权威的年级视角参考。"""


    user_prompt = f"""请基于以下信息，生成详实、专业的《年级适配性评估报告》：

**作文基本信息:**
*   年级: {grade}
*   题目要求 (参考): {requirement}...
*   识别的文体类型: {essay_type or '未知'}
*   **预设整体分类**: {predicted_classification}

请严格遵循系统提示中的三个评估维度和输出结构，确保评估内容紧密围绕切题度与{grade}年级标准的适配性，输出《年级适配性评估报告》。"""

    return [Message(role="system", content=system_prompt), Message(role="user", content=user_prompt)]

# --- 新 Agent B: 写作技巧与表达方式分析器 ---
def generate_technique_analyzer_prompt_B(content: str, essay_type: Optional[str]) -> List[Message]:
    """为新 Agent B (写作技巧与表达方式分析器) 生成 Prompt"""
    system_prompt = f"""角色: 资深语文评阅专家，擅长分析文本的写作技巧和表达方式，对各学龄段学生作文的切题度评估有丰富经验。
任务: 深入分析给定作文的全文内容，识别并评估其中运用的主要写作技巧、表达方式和修辞手法，**重点判断这些技巧是否有效促进作文紧扣题目要求、准确传达中心思想，或在何种程度上偏离了题目核心**。

分析要点与输出结构 (请严格按此格式输出《写作技巧分析报告》):

1.  **主要表达方式运用 (Primary Expression Methods):**
    *   精准识别文章主要运用了哪些表达方式（记叙、描写、抒情、说明、议论）及其比例分布？
    *   **这些表达方式是否与题目隐含的写作任务相匹配**？它们与识别的文体类型（{essay_type or '未知'}）及文章主旨的契合度如何？请分析它们如何共同服务于中心思想的表达与题目要求的回应。

2.  **关键写作技巧/修辞手法识别 (Key Techniques/Rhetoric):**
    *   系统梳理文章中使用的较为突出或关键的写作技巧或修辞手法（例如：细节描写、比喻、拟人、排比、对比、象征、首尾呼应、设置悬念、引用等）。
    *   选取最能体现作者切题意识的1-2个技巧，引用原文具体例子进行分析。

3.  **技巧对切题度/中心思想的影响评估 (Impact on Relevance/Theme):**
    *   **核心评估点**: **这些被识别的技巧和表达方式，在多大程度上强化了作文与题目要求的关联性、增强了中心思想的清晰度和突出度**？或者，它们在哪些方面可能削弱了作文的切题表现（例如：过度修饰性描写导致主题模糊，议论过于泛化而缺乏针对性，叙事脱离主题等）？
    *   请具体量化分析各项技巧运用与"切题/突出中心"目标之间的关系（显著促进/一般促进/关联不大/存在干扰），并说明原因。

请确保分析聚焦于技巧本身及其**对内容切题度和中心思想表达的具体影响**，避免对技巧运用水平本身进行价值评判（不使用"用得好/不好"等主观评价），而是客观分析其对切题度的实际效果。分析语言应专业、精准、中立。"""

    user_prompt = f"""请基于以下作文内容和文体类型信息，生成《写作技巧分析报告》：

**识别的文体类型:** {essay_type or '未知'}

**作文全文内容 (Content):**
{content}

请严格按照系统提示中的分析要点和输出结构，输出《写作技巧分析报告》。着重评估写作技巧对作文切题度的影响。"""

    return [Message(role="system", content=system_prompt), Message(role="user", content=user_prompt)]

# --- 新 Agent C: 素材选择与应用评估器 ---
def generate_material_assessor_prompt_C(content: str, requirement: str, essay_type: Optional[str]) -> List[Message]:
    """为新 Agent C (素材选择与应用评估器) 生成 Prompt"""
    system_prompt = f"""角色: 专业的文本素材评估专家，精通教育写作评估与文体分析。
任务: 全面审视作文内容，结合题目要求和文体特征，深入评估其中选用的**素材（如：具体事例、论据支撑、数据引证、经典引用、描写场景、人物塑造等）**的适切性、支撑力度与题旨匹配度。

分析要点与输出结构 (请严格按此格式输出《素材评估报告》):

1.  **主要素材识别 (Identification of Key Materials):**
    *   精准识别文章中用于支撑核心观点、展开关键叙述或进行主题说明的核心素材。（简要列举1-3个最具代表性的素材元素）
2.  **素材与题目要求的相关性 (Relevance to Prompt Requirements):**
    *   系统评估所选素材是否精准对应题目要求（含主题范围、写作任务、特定限定条件等）？（评级：高度相关/基本相关/部分相关/关联度低）
    *   精炼阐述判断依据，尤其分析素材是否直接回应题目核心要点和隐含要求。
3.  **素材对中心思想的支持度 (Support for Central Idea):**
    *   深度评估这些素材在论证观点深度、阐释主旨清晰度、深化主题意蕴方面的实际效能。它们是否构成文章中心思想的有力支柱？（评级：支撑有力/基本支撑/支撑不足/与主旨关联弱或无关）
    *   精确剖析素材与中心思想间的逻辑关联与呼应机制。
4.  **素材与文体类型的协调性 (Consistency with Genre):**
    *   所选素材的性质、层次与呈现方式是否高度契合当前文体类型（{essay_type or '未知'}）的规范要求？（如：议论文素材是否具备充分说服力与典型性？记叙文素材是否生动具体且情景完整？说明文素材是否准确清晰且条理分明？）

请确保评估聚焦于素材的**相关性、有效性和恰当性**三个核心维度，尤其关注素材如何精准服务于**回应题目要求和支撑中心思想**这两大关键目标。避免评价素材的趣味性或新颖度等次要特征。保持评估语言客观、精准、专业。"""
    user_prompt = f"""请基于以下作文内容、题目要求和文体类型信息，生成《素材评估报告》：

**题目要求 (Requirement):**
{requirement}

**识别的文体类型:** {essay_type or '未知'}

**作文全文内容 (Content):**
{content}

请严格按照系统提示中的分析要点和输出结构，输出《素材评估报告》。"""
    return [Message(role="system", content=system_prompt), Message(role="user", content=user_prompt)]

# --- 新 Agent D: 结构逻辑与层次分析器 ---
def generate_structure_analyzer_prompt_D(content: str, essay_type: Optional[str]) -> List[Message]:
    """为新 Agent D (结构逻辑与层次分析器) 生成 Prompt"""
    system_prompt = f"""角色: 精通篇章分析的逻辑学家，擅长解构文章的组织结构和论证层次。
任务: 分析作文的整体结构设计、段落安排、逻辑顺序和层次推进，评估文章的结构**是否有助于清晰地表达中心思想、紧密地围绕主题展开，并有效地回应题目要求**。

分析要点与输出结构 (请严格按此格式输出《结构逻辑分析报告》):

1.  **整体结构模式识别 (Overall Structural Pattern):**
    *   识别文章采用了什么样的整体结构模式？（例如：总分总、并列式、递进式、时间顺序、空间顺序、对比式等，或结构不够清晰）
    *   该结构模式是否适合表达本文的中心思想和回应题目要求？是否符合文体类型（{essay_type or '未知'}）的常见结构？
2.  **段落功能与衔接 (Paragraph Function & Cohesion):**
    *   评估主要段落（或部分）的功能是否明确（如：提出观点、阐述理由、举例说明、过渡、总结等）？
    *   段落之间的衔接是否自然、过渡是否流畅？逻辑关系是否清晰？
3.  **逻辑层次与推进 (Logical Hierarchy & Progression):**
    *   文章内容的展开是否有清晰的逻辑层次？论证或叙述是否层层递进、逐步深入？
    *   是否存在逻辑跳跃、层次混乱或重点不明的问题？
4.  **结构对切题度/中心思想的影响评估 (Impact on Relevance/Theme):**
    *   **核心评估点**: 文章的结构安排在多大程度上**促进**了内容的聚焦、主旨的突出和对题目要求的有效回应？或者在哪些方面**阻碍**了这些目标的实现（例如：结构松散导致主题涣散，缺乏过渡导致逻辑不清，重点段落安排不当导致中心不明）？
    *   请具体分析结构与"切题/突出中心"目标之间的关系（正向促进/影响不大/存在干扰）。

请确保分析聚焦于结构本身及其**对内容核心表达（切题度和中心思想）的功能作用**。避免评价结构本身的优劣（不说"结构好/不好"），只分析其效果。语言客观、中立。"""
    user_prompt = f"""请基于以下作文内容和文体类型信息，生成《结构逻辑分析报告》：

**识别的文体类型:** {essay_type or '未知'}

**作文全文内容 (Content):**
{content}

请严格按照系统提示中的分析要点和输出结构，输出《结构逻辑分析报告》。"""

    return [Message(role="system", content=system_prompt), Message(role="user", content=user_prompt)]


COMMENT_STRUCTURE_GUIDELINES = {
    "优秀": """
【优秀作文评语框架】
• 开篇部分：先概括性地评价文章主题与题目要求的高度契合度，点明文章的整体风格与特点
• 中心分析：分层次详细分析作文如何准确把握题目核心要求，在哪些方面展现出突出表现（如立意深刻、结构严谨、内容丰富、角度新颖等）
• 亮点提炼：从多个维度（如语言表达、情感表达、思想深度、逻辑构建等）具体举例分析文章的优秀之处，引用文中精彩片段进行说明
• 小建议部分：即使是优秀作文，也可提出一两点更上层楼的微小建议，如某处表达可更精准，某个观点可更深入等
• 总结评价：以“总体来看”，“总的来说”，“总体而言”，“总而言之”等等开头但不要固定在这四个里，总结评价作文的切题表现，重申文章与题目的高度契合以及中心思想的突出表达（最后一句总结评价切题度等文章内容，并且禁止提建议）

评语应当深入、全面、有理有据，通过具体分析让作者清晰了解自己的优势所在，同时感受到评语者的用心与专业，总体约190-220字。
""", # <-- 修改长度提示

    "较好": """
【较好作文评语框架】
• 开篇认可：肯定文章基本符合题目要求，指出文章的总体方向和风格特点
• 优点分析：从多角度详细分析文章在符合题目要求方面的具体表现，如主题把握、内容选择、结构安排等方面的成功之处
• 问题指出：委婉指出文章在切题程度或中心表达上存在的不足，如某些部分可与主题联系更紧密，某些观点可发掘更深等
• 改进建议：针对上述不足，提出具体、可操作的改进建议，包括如何调整内容重点、如何强化中心思想表达、如何使结构更加合理等
• 延伸思考：引导作者思考如何将当前水平提升至更高层次，提出拓展性问题或建议
• 总结评价：以“总体来看”，“总的来说”，“总体而言”，“总而言之”等等开头，总结评价作文的整体表现，再次确认文章的切题表现（其切题水平为较好），是否跑题，总结优缺点（最后一句总结评价切题度等文章内容，并且禁止提建议）

评语应当平衡肯定与建议，既让作者感受到成就，又明确指出提升空间，语言要具体而不空泛，分析要深入而不流于表面，总体约190-220字。
""", # <-- 修改长度提示

    "一般": """
【一般作文评语框架】
• 基本评价：客观指出文章与题目要求之间的关系，认可其基本回应了题目要求，但存在明显不足
• 切题分析：详细分析文章在哪些方面基本符合题目要求，又在哪些方面偏离或理解不够到位
• 中心问题：重点剖析文章在主题把握、中心表达方面的主要问题，如主旨不明确、内容散乱、重点不突出等
• 具体实例：引用文中的具体段落或句子，说明其与题目要求的关联度不足或对中心思想的支撑不够
• 深入建议：提出多方面、有针对性的改进建议，包括如何重新理解题目要求、如何调整内容结构、如何强化中心思想等
• 写作方法：结合文章实际，提供一些实用的写作技巧和方法，帮助作者提升表达能力
• 总结评价：以“总体来看”，“总的来说”，“总体而言”，“总而言之”等等开头，总结评价作文的整体表现，再次确认文章的切题表现（其切题水平为一般），是否跑题，总结优缺点（最后一句总结评价切题度等文章内容，并且禁止提建议）

评语应当客观公正，既不过分批评打击信心，也不回避问题，通过具体分析帮助作者清晰认识到作文的不足之处，总体约190-220字。
""", # <-- 修改长度提示

    "合格": """
【合格作文评语框架】
• 直接指出：开门见山地指出文章与题目要求之间存在的明显偏差，但肯定其基本达到了合格水平
• 问题分类：系统分析文章在理解题意、立意、内容选择、结构安排等方面存在的各类问题
• 偏题分析：详细阐述文章在哪些方面、为什么偏离了题目的核心要求，或者对题目的理解有何误区
• 具体举例：选取文中的多个具体部分，分析其如何偏离主题或未能有效表达中心思想
• 审题建议：强调正确审题的重要性，提供具体的审题方法和步骤，帮助作者在今后写作中避免类似问题
• 写作指导：针对文章的具体问题，提供相应的写作指导和建议，如如何确立中心、如何选取材料、如何组织结构等
• 修改方向：若对当前文章进行修改，应从哪些方面入手，需要作出哪些调整才能更好地符合题目要求
• 总结评价：以“总体来看”，“总的来说”，“总体而言”，“总而言之”等等开头，总结评价作文的整体表现，再次确认文章的切题表现（其切题水平为合格），是否偏离题目要求，总结优缺点（最后一句总结评价切题度等文章内容，并且禁止提建议）
评语应当在指出问题的同时注重建设性意见，语言要诚恳、具体、有指导性，避免空泛的批评，总体约190-220字。
""", # <-- 修改长度提示

   "不合格": """
   【不合格作文评语框架】
• 直接定性：开门见山地指出作文切题和中心不合格，明确表明不符合题目要求的具体方面。
• 具体偏差：清晰列出作文与题目要求之间的具体偏差，提出关键问题。
• 原因分析：简明扼要地分析导致跑题的原因。
• 写作建议：提供简短、直接的写作建议，帮助作者避免再次跑题。
评语应当实事求是但不过分苛责，重点放在如何帮助作者理解并避免再次出现类似问题上，语言要诚恳、具体、富有指导性，总体约190-220字。
评语的总体结构应该是：
首先直接指出作文不合格的原因（切题和中心方面的问题）
然后详细说明作文偏离题目要求的具体表现
简明扼要地分析导致跑题的原因。
最后给出明确的写作建议
注意：总结评价：不合格的作文严禁使用总结评价，不要使用"总体来看"，"总的来说"，"总体而言"，"总而言之"等总结性质的文字。评语应该简洁明了，直击要害，避免冗长的分析和过多的细节。

""", # <-- 修改长度提示

    "默认": """
【通用评语框架】
• 整体评估：首先对作文与题目要求的契合度及中心思想表达进行整体评估
• 内容分析：从多个维度详细分析作文内容与题目要求的关联程度，包括主题把握、内容选择、结构安排等
• 亮点识别：找出作文中与题目要求紧密相关的亮点部分，并具体说明其优点
• 不足剖析：客观指出作文在切题程度或中心表达上的不足之处，并分析原因
• 具体建议：针对不足提出具体、可操作的改进建议，包括如何更好地理解题目、调整内容重点、强化中心思想等
• 写作指导：结合作文实际，提供提升切题能力和中心表达的写作技巧和方法
• 总结期望：以“总体来看”，“总的来说”，“总体而言”，“总而言之”等等开头，总结评价作文的整体表现，表达对作者进步的期望（最后一句总结评价切题度等文章内容，并且禁止提建议）

评语应当全面、客观、具体、有针对性，既要指出问题，也要给予建设性意见。评语应通过多角度分析、多维度评价和具体事例引用等方式，确保内容丰富、深入且有实际指导价值，总体约190-220字。
""" # <-- 修改长度提示
}


# Agent 4: 初步评语生成器 (修改长度要求)
def generate_draft_comment_generator_prompt(
    essay_data: Dict[str, Any],
    predicted_classification: str,
    essay_type: Optional[str],
    prompt_analysis: str,
    theme_analysis: str,
    grade_adaptability_analysis: str, # Agent A
    technique_analysis: str,          # Agent B
    material_analysis: str,           # Agent C
    structure_analysis: str,          # Agent D
    formatted_samples: str
) -> List[Message]:

    grade = essay_data.get("grade", "指定")
    title = essay_data.get("title", "")
    type_info = f"背景信息：预设文体类型倾向于“{essay_type}”。" if essay_type else ""
    sample_info = formatted_samples
    structure_guideline = COMMENT_STRUCTURE_GUIDELINES.get(predicted_classification, COMMENT_STRUCTURE_GUIDELINES["默认"])

    # 修改点: 长度要求改为 160-200
    system_prompt = f"""你是一位经验丰富、表达精准且富有启发性的中小学特级作文评审导师。
当前任务: 为一篇{grade}年级的学生作文，基于前面多位专家提供的深度分析报告，并严格依据预设的作文整体分类：{predicted_classification}，撰写一条**高度聚焦于“切题度”和“中心思想”**的评语草稿。
{type_info}

输入信息汇总:
*   作文基本信息：年级({grade}), 标题({title}), 文体({essay_type or '未识别'})
*   题目要求分析 (Agent 1): 理解显性要求。
*   作文主旨分析 (Agent 2): 把握实际内容和中心。
*   年级适配性评估 (Agent A): 考量切题表现是否符合年级。
*   写作技巧分析 (Agent B): 技巧对主题表达的影响。
*   素材选择评估 (Agent C): 素材对主题支持的相关性、有效性。
*   结构逻辑分析 (Agent D): 结构对主题表达的影响。
*   **核心依据**: 预设作文分类: **{predicted_classification}** (你的评语基调和核心判断必须与此分类严格一致！)

评语撰写核心要求与规则（请严格遵守）：

1.  **绝对主题聚焦**: 评语的每一个字都必须直接关联作文的“切题性”（内容、中心是否紧扣题目要求）和“中心思想”（是否明确、突出、与题目契合）。
2.  **利用分析解释表现**: 可以**适度利用** Agent B, C, D 的分析结果（技巧、素材、结构对切题/中心的影响），来**具体解释**为什么文章的切题度或中心思想表达会达到 {predicted_classification} 的水平（无论是好是坏）。例如，可以说“清晰的段落结构有助于紧扣中心论述”，或者“部分素材的选择未能有效支撑主旨，导致中心不够突出”。
3.  **严禁独立评价**: 绝对禁止脱离“切题度/中心思想”这个核心，去独立评价语言文字（文采、词句）、写作技巧本身（不说“比喻用得好”）、素材选择（不说“例子很生动”）、结构安排（不说“结构完整”）、创意构思、情感表达方式、书写规范、篇幅长短等。**所有提及技巧/素材/结构的文字，都必须落脚到其对“切题/中心”的影响上。**
4.  **结论与分析一致**: 评语的整体评价基调、指出的优缺点，必须与输入的 **{predicted_classification}** 分类以及 Agent A, B, C, D 的分析报告（尤其是关于影响的评估）高度吻合。
5.  **建设性建议 (若非优秀)**:
    *   如果分类是“较好”、“一般”或“合格”，必须针对性地提出 **1 条** 具体的、可操作的、**仅关于如何改进“切题性”或“中心思想贴合度”** 的建议。建议应基于分析报告发现的核心问题（可能涉及审题、立意、内容组织、素材调整等，但最终目的是提升切题度）。
    *   如果分类是“不合格”，建议聚焦于**如何正确审题和确立基本方向**。
    *   如果分类是“优秀”，**禁止提任何建议**。
    *   建议必须原创，严禁照搬示例。
6.  **专业语言与语气**: 使用中小学语文教师评语中常见的专业术语（仅限与切题度和中心思想及其影响因素相关的）。语气客观、中肯、符合教师身份，考虑{grade}年级学生接受度。
7.  **格式与长度**: 输出纯文本评语，不含特殊符号、Markdown、引号（书名号除外）、列表、换行符。长度严格控制在 **190-220 个字符之间**（必须不少于 190）。

结构指导参考:
请在遵循上述所有规则的前提下，参考以下针对 **{predicted_classification}** 类作文的【评语结构建议】来组织评语内容：
{structure_guideline}
**请特别注意**：【评语结构建议】仅供启发思考和组织逻辑。**严禁直接复制、照搬或简单套用指导中的任何原话或句子结构**。 你必须用自己的语言，综合所有输入信息，围绕切题度和中心思想（及其影响因素），独立构思并表达评语内容，确保评语自然、流畅且具有原创性。评语长度必须不少于 190 字符。

{sample_info}

请开始工作，生成符合要求的评语草稿。"""

    # 修改点: user prompt 中的长度说明
    user_prompt = f"""请为这篇{grade}年级、标题为《{title}》的作文，根据系统提示中提供的所有分析报告和预设分类（{predicted_classification}），生成一条仅关于切题度和中心思想的评语草稿。

关键分析输入回顾 (片段):
*   题目要求分析: {prompt_analysis}...
*   作文主旨分析: {theme_analysis}...
*   年级适配性: {grade_adaptability_analysis}...
*   技巧影响分析: {technique_analysis}...
*   素材影响分析: {material_analysis}...
*   结构影响分析: {structure_analysis}...
*   **核心指令: 最终评语必须严格符合 {predicted_classification} 的定性。**

请严格遵循系统提示中的所有规则（特别是主题聚焦、利用分析解释、严禁独立评价、建议针对性、与分类一致、专业语言、格式与长度190-220字符），并参考提供的【{predicted_classification}作文评语结构建议】来组织语言，生成评语草稿（必须不少于190字符）。"""
    return [Message(role="system", content=system_prompt), Message(role="user", content=user_prompt)]

# Agent 5: 评语合规与润色器 (修改长度要求)
def generate_compliance_checker_prompt(
    draft_comment: str,
    predicted_classification: str,
    essay_type: Optional[str],
    formatted_samples: str
) -> List[Message]:

    type_info = f"背景信息：预设文体类型倾向于“{essay_type}”。" if essay_type else ""
    sample_info = formatted_samples
    structure_guideline = COMMENT_STRUCTURE_GUIDELINES.get(predicted_classification, COMMENT_STRUCTURE_GUIDELINES["默认"])

    # 修改点: 长度要求改为 160-200
    system_prompt = f"""你是一位极其严谨、注重细节的中小学作文评语终审专家，扮演着质量控制的最后一道关卡。
任务: 对提供的作文评语草稿进行全面、彻底的合规性检查和必要的精炼润色，确保最终输出的评语完美无瑕地满足所有既定规则。
{type_info}

最终评语必须严格遵守以下规则清单（请逐条核对并修正）：

1.  **【核心原则】聚焦切题度与中心思想**:
    *   检查评语是否**始终聚焦**于讨论作文的“切题性”（内容、中心是否紧扣题目要求）和“中心思想的表达与切合度”。
    *   **允许**基于之前的分析，适度提及写作技巧、素材运用或篇章结构**对切题度和中心思想表达产生的具体影响**（例如：“生动的描写有助于更好地展现主题”、“清晰的结构使文章能紧扣中心展开论述”、“部分素材的选择未能有效支撑主旨”）。
    *   **绝对禁止**对语言运用（词语、句子、修辞、文采）、写作技巧本身、素材选择本身（趣味性、新颖度）、结构安排本身（完整性）、创意构思、情感表达方式、错别字、标点、篇幅等进行**脱离“切题度/中心思想”影响的独立评价或描述**。这是最重要的规则。
2.  **【内容安全】合规性**: 确保评语内容健康、积极，绝不包含敏感、不当或违法违规信息。
3.  **【格式规范】纯文本**: 确保评语为单一段落的纯文本。清除所有特殊符号（除标准中文标点和必要书名号《》）、Markdown、HTML、列表标记、非书名号的引号（"" ''）、不必要括号、换行符。前后无多余空格。书名号仅酌情使用。
4.  **【长度硬性约束】字数控制**: 最终字符数（含标点）**必须在 190-220 个字符之间**。
    *   若草稿 < 160，在不违反规则1的前提下，扩展对切题度/中心思想（或其影响因素）的描述或建议，使其达到190，但不超220。
    *   若草稿 > 200，精简语言，优先保留核心评价与建议，确保**截断到220字符以内**。
5.  **【一致性校验】口径统一**: 再次确认评语基调、评价等级和核心问题，是否与参考的“预测分类结果”({predicted_classification})完全一致？不允许矛盾。
6.  **【结构参考】逻辑层次**: 检查评语内部逻辑是否清晰、表达是否流畅。可参考以下【评语结构建议】优化层次感：
    {structure_guideline}
    **注意**：结构建议仅供参考，**严禁直接复制、照搬或简单套用其中语句**。最终评语需自然原创。
7.  **【语言质量】专业流畅**: 在满足规则前提下，润色语言，确保表达准确、自然、流畅，符合中小学语文教师专业风格。术语使用恰当（仅限切题与主旨相关及其影响因素）。
8.  **【总结句规范】**（仅适用于“优秀”、“较好”、“一般”、“合格”，**“不合格”严禁使用**）：
    *   最后一句话应为总结性评价。
    *   必须以总结性词语开头（如“总体来看”、“总的来说”等，**从中随机选择一个**）。
    *   必须再次点明文章的切题表现或核心问题，与 {predicted_classification} 分类一致。
    *   **总结句中绝对禁止再提任何新的建议或期望**。

最终输出:
你的输出**只能是**经过上述严格检查和必要润色后，完全符合所有规则（特别是规则1、3、4、5、8）的最终评语文本本身。**严禁包含任何解释、说明、标题、标签或评语之外的文字**。最终评语长度必须在 190-220 字符区间内。

请开始审核与处理。
{sample_info}
"""
    # 修改点: user prompt 中的长度说明
    user_prompt = f"""请严格按照上述系统提示中的所有规则清单，对以下作文评语草稿进行终审、修正和润色。确保输出完全合规的最终评语。

**待审评语草稿：**
{draft_comment}

**参考信息：**
*   预测分类结果： {predicted_classification}
*   (作文类型：{essay_type or '未知'})

请直接输出符合所有规则（尤其是规则1聚焦影响、规则3纯文本、规则4长度190-220、规则5与分类一致、规则8总结句规范）的、纯文本最终评语。"""
    return [Message(role="system", content=system_prompt), Message(role="user", content=user_prompt)]


# --- 主要处理函数 (修改长度校验) ---
def generate_essay_comment(
    essay_data: Dict[str, Any],
    classification_results: Dict[str, str],
    samples_data: List[Dict[str, Any]],
    processed_comments_cache: Dict[str, str]
) -> Optional[Dict[str, Any]]:
    """为单篇作文生成评论 - 包含新 Agent 流程和动态样本选择"""
    essay_id_track2 = essay_data.get("id")
    essay_id_track1 = essay_data.get("track1_id")
    essay_id_track2_str = str(essay_id_track2)
    essay_id_track1_str = str(essay_id_track1) if essay_id_track1 is not None else None

    if essay_id_track1 is None:
        logger.error(f"[Track2 ID: {essay_id_track2_str}] 缺少 Track 1 ID。")
        return {"id": essay_id_track1, "comment": "ERROR: Missing Track 1 ID mapping"}

    start_time = time.time()
    logger.info(f"[Track1 ID: {essay_id_track1_str}, Track2 ID: {essay_id_track2_str}] 开始处理...")

    # 1. 检查缓存 (修改点: 校验长度 120-190)
    if essay_id_track2_str in processed_comments_cache:
        cached_comment = processed_comments_cache[essay_id_track2_str]
        if isinstance(cached_comment, str) and 120 <= len(cached_comment) <= 190:
            logger.info(f"[Track1 ID: {essay_id_track1_str}] 评论从缓存加载 (Key: {essay_id_track2_str})。")
            return {"id": essay_id_track1, "comment": cached_comment}
        else:
            # 修改点: 更新日志中的长度信息
            logger.warning(f"[Track1 ID: {essay_id_track1_str}] 缓存评论无效 (长度 {len(cached_comment)}，要求 [120, 190])，重新生成。")
            if essay_id_track2_str in processed_comments_cache: # Check again before deleting
                 del processed_comments_cache[essay_id_track2_str]

    # 2. 获取分类
    predicted_classification = classification_results.get(essay_id_track1_str, "一般")
    logger.info(f"[Track1 ID: {essay_id_track1_str}] 预设分类: {predicted_classification}")

    # 3. 格式化样本
    formatted_samples = format_samples_for_prompt(samples_data, predicted_classification)
    log_level = logging.DEBUG if formatted_samples else logging.WARNING
    logger.log(log_level, f"[Track1 ID: {essay_id_track1_str}] 分类 '{predicted_classification}' 的样本格式化情况: {'找到' if formatted_samples else '未找到'}。")

    # 4. 执行 Agent 链
    max_retries = 100
    retry_count = 0
    final_comment = ""
    # 初始化所有分析结果变量
    essay_type = None
    prompt_analysis = "N/A"
    theme_analysis = "N/A"
    grade_adaptability_analysis = "N/A" # Agent A
    technique_analysis = "N/A"          # Agent B
    material_analysis = "N/A"           # Agent C
    structure_analysis = "N/A"          # Agent D
    error_message = ""

    while retry_count < max_retries:
        agent_chain_successful = True
        api_error_occurred = False
        current_error = "" # Track error in this attempt

        try:
            # --- 前期分析 ---
            # Agent 0: 类型识别
            if not essay_type:
                logger.debug(f"[{essay_id_track1_str}] Agent 0: 识别类型...")
                essay_type_classifier_messages = generate_essay_type_classifier_prompt(essay_data)
                essay_type_response = call_llm_api(essay_type_classifier_messages, agent_name=f"Agent0_Type_{essay_id_track1_str}")
                if "API_ERROR" in str(essay_type_response):
                    logger.warning(f"[{essay_id_track1_str}] Agent 0 错误: {essay_type_response}")
                    essay_type = "记叙文" # Default on error
                    api_error_occurred = True
                    current_error = essay_type_response
                else:
                    essay_type = extract_essay_type(essay_type_response)
                logger.info(f"[{essay_id_track1_str}] 识别类型: {essay_type}")

            # Agent 1: 题目解析
            if prompt_analysis == "N/A" and not api_error_occurred: # Proceed if no error so far
                logger.debug(f"[{essay_id_track1_str}] Agent 1: 解析题目...")
                prompt_analyzer_messages = generate_prompt_analyzer_prompt(essay_data, essay_type)
                prompt_analysis = call_llm_api(prompt_analyzer_messages, agent_name=f"Agent1_Prompt_{essay_id_track1_str}")
                if "API_ERROR" in str(prompt_analysis):
                    logger.warning(f"[{essay_id_track1_str}] Agent 1 错误: {prompt_analysis}")
                    agent_chain_successful = False
                    api_error_occurred = True
                    current_error = prompt_analysis

            # Agent 2: 主旨提取
            if theme_analysis == "N/A" and agent_chain_successful:
                logger.debug(f"[{essay_id_track1_str}] Agent 2: 提取主旨...")
                theme_extractor_messages = generate_essay_theme_extractor_prompt(essay_data, essay_type)
                theme_analysis = call_llm_api(theme_extractor_messages, agent_name=f"Agent2_Theme_{essay_id_track1_str}")
                if "API_ERROR" in str(theme_analysis):
                    logger.warning(f"[{essay_id_track1_str}] Agent 2 错误: {theme_analysis}")
                    agent_chain_successful = False
                    api_error_occurred = True
                    current_error = theme_analysis

            # --- 深度分析 ---
            # Agent A: 年级适配性
            if grade_adaptability_analysis == "N/A" and agent_chain_successful:
                logger.debug(f"[{essay_id_track1_str}] Agent A: 评估年级适配性...")
                adaptability_messages = generate_grade_adaptability_assessor_prompt_A(essay_data, essay_type, predicted_classification)
                grade_adaptability_analysis = call_llm_api(adaptability_messages, agent_name=f"AgentA_Adapt_{essay_id_track1_str}")
                if "API_ERROR" in str(grade_adaptability_analysis):
                    logger.warning(f"[{essay_id_track1_str}] Agent A 错误: {grade_adaptability_analysis}")
                    agent_chain_successful = False
                    api_error_occurred = True
                    current_error = grade_adaptability_analysis

            # Agent B: 技巧分析
            if technique_analysis == "N/A" and agent_chain_successful:
                logger.debug(f"[{essay_id_track1_str}] Agent B: 分析技巧影响...")
                technique_messages = generate_technique_analyzer_prompt_B(essay_data.get("content", ""), essay_type)
                technique_analysis = call_llm_api(technique_messages, agent_name=f"AgentB_Technique_{essay_id_track1_str}")
                if "API_ERROR" in str(technique_analysis):
                    logger.warning(f"[{essay_id_track1_str}] Agent B 错误: {technique_analysis}")
                    agent_chain_successful = False
                    api_error_occurred = True
                    current_error = technique_analysis

            # Agent C: 素材评估
            if material_analysis == "N/A" and agent_chain_successful:
                logger.debug(f"[{essay_id_track1_str}] Agent C: 评估素材影响...")
                material_messages = generate_material_assessor_prompt_C(essay_data.get("content", ""), essay_data.get("requirement", ""), essay_type)
                material_analysis = call_llm_api(material_messages, agent_name=f"AgentC_Material_{essay_id_track1_str}")
                if "API_ERROR" in str(material_analysis):
                    logger.warning(f"[{essay_id_track1_str}] Agent C 错误: {material_analysis}")
                    agent_chain_successful = False
                    api_error_occurred = True
                    current_error = material_analysis

            # Agent D: 结构分析
            if structure_analysis == "N/A" and agent_chain_successful:
                logger.debug(f"[{essay_id_track1_str}] Agent D: 分析结构影响...")
                structure_messages = generate_structure_analyzer_prompt_D(essay_data.get("content", ""), essay_type)
                structure_analysis = call_llm_api(structure_messages, agent_name=f"AgentD_Structure_{essay_id_track1_str}")
                if "API_ERROR" in str(structure_analysis):
                    logger.warning(f"[{essay_id_track1_str}] Agent D 错误: {structure_analysis}")
                    agent_chain_successful = False
                    api_error_occurred = True
                    current_error = structure_analysis

            # --- 评语生成 ---
            # Agent 4: 初步评语
            if agent_chain_successful: # Only generate draft if all analyses were successful
                logger.debug(f"[{essay_id_track1_str}] Agent 4: 生成初步评语...")
                draft_generator_messages = generate_draft_comment_generator_prompt(
                    essay_data, predicted_classification, essay_type,
                    prompt_analysis, theme_analysis,
                    grade_adaptability_analysis, technique_analysis,
                    material_analysis, structure_analysis,
                    formatted_samples
                )
                draft_comment = call_llm_api(draft_generator_messages, agent_name=f"Agent4_Draft_{essay_id_track1_str}")
                if "API_ERROR" in str(draft_comment):
                    logger.warning(f"[{essay_id_track1_str}] Agent 4 错误: {draft_comment}")
                    agent_chain_successful = False
                    api_error_occurred = True
                    current_error = draft_comment

            # Agent 5: 合规润色
            if agent_chain_successful: # Only finalize if draft was generated
                logger.debug(f"[{essay_id_track1_str}] Agent 5: 检查并润色...")
                compliance_checker_messages = generate_compliance_checker_prompt(
                    draft_comment, predicted_classification, essay_type, formatted_samples
                )
                final_comment_raw = call_llm_api(compliance_checker_messages, agent_name=f"Agent5_Compliance_{essay_id_track1_str}")
                if "API_ERROR" in str(final_comment_raw):
                    logger.warning(f"[{essay_id_track1_str}] Agent 5 错误: {final_comment_raw}")
                    agent_chain_successful = False
                    api_error_occurred = True
                    current_error = final_comment_raw
                else:
                    final_comment = clean_comment(final_comment_raw)
                    logger.debug(f"[{essay_id_track1_str}] 清理后最终评论: {final_comment}")
                    comment_len = len(final_comment)
                    # 修改点: 最终校验长度 120-190
                    if 120 <= comment_len <= 190:
                        logger.info(f"[{essay_id_track1_str}] 成功生成合规评论 (长度 {comment_len})。")
                        break # Success, exit retry loop
                    else:
                        # 修改点: 更新日志中的长度信息
                        logger.warning(f"[{essay_id_track1_str}] Agent 5 评论长度不合规 ({comment_len}，要求 [120, 190])，重试。")
                        agent_chain_successful = False # Mark for retry
                        final_comment = "" # Reset comment for retry

            # --- 重试逻辑 ---
            if not agent_chain_successful or api_error_occurred:
                retry_count += 1
                error_message = current_error # Store the last error message for logging
                logger.warning(f"[{essay_id_track1_str}] Agent 链失败或评论不合规 (错误: {error_message})，准备重试 {retry_count}/{max_retries}...")
                time.sleep(1.5 ** retry_count + random.uniform(0,1))
                # Reset states for failed steps and subsequent ones
                if api_error_occurred:
                    if error_message == essay_type_response: essay_type = None
                    if error_message == prompt_analysis or prompt_analysis == "N/A": prompt_analysis = "N/A" # Reset if failed or not reached
                    if error_message == theme_analysis or theme_analysis == "N/A": theme_analysis = "N/A"
                    if error_message == grade_adaptability_analysis or grade_adaptability_analysis == "N/A": grade_adaptability_analysis = "N/A"
                    if error_message == technique_analysis or technique_analysis == "N/A": technique_analysis = "N/A"
                    if error_message == material_analysis or material_analysis == "N/A": material_analysis = "N/A"
                    if error_message == structure_analysis or structure_analysis == "N/A": structure_analysis = "N/A"
                # Always reset final comment if retry is needed
                final_comment = ""
                continue # Continue to the next retry attempt

        except Exception as e:
            logger.error(f"[{essay_id_track1_str}] 生成评论意外错误: {str(e)}", exc_info=True)
            retry_count += 1
            logger.info(f"[{essay_id_track1_str}] 准备重试 {retry_count}/{max_retries}...")
            time.sleep(1.5 ** retry_count + random.uniform(0,1))
            # Reset all states on unexpected error
            final_comment = ""
            essay_type = None
            prompt_analysis = "N/A"
            theme_analysis = "N/A"
            grade_adaptability_analysis = "N/A"
            technique_analysis = "N/A"
            material_analysis = "N/A"
            structure_analysis = "N/A"
            error_message = str(e) # Log the exception as the error

    # 5. 检查最终结果和备用逻辑 (修改点: 校验长度 120-190, 备用长度 120-190)
    if not (isinstance(final_comment, str) and 120 <= len(final_comment) <= 190):
        logger.error(f"[{essay_id_track1_str}] 达最大重试或最终评论仍不合格 (长度 {len(final_comment)})，生成备用评语。最后错误: {error_message}")
        title = essay_data.get("title", "这篇作文")
        grade = essay_data.get("grade", "该年级")
        backup_comment = f"本文基于{grade}年级要求，对《{title}》的分析表明其整体表现评定为“{predicted_classification}”。"
        if predicted_classification == "优秀":
            backup_comment += "文章精准把握题意，中心思想突出，内容高度聚焦题目要求，展现了良好的审题和构思能力。"
        elif predicted_classification == "较好":
            backup_comment += "文章较好回应题目核心要求，中心明确，内容主体紧扣主题。细节或深度略有不足。建议思考如何让主题表达更凝练。"
        elif predicted_classification == "一般":
            backup_comment += "文章大致围绕题目展开，但对题意理解不够深入，中心思想不够突出，部分内容与主题联系需加强。建议审题时更侧重核心要求。"
        elif predicted_classification == "合格":
            backup_comment += "文章在把握题目要求方面明显不足，中心思想模糊或偏离，内容契合度不高。建议重审题意，明确写作方向和中心。"
        else: # 不合格
            backup_comment += "文章未能达到题目基本要求，内容严重偏离主题，中心思想不清或与题目无关。务必在写作前仔细审题。"

        # 修改点: 截断到 190
        final_comment = backup_comment[:190]
        # 修改点: 填充到 120
        if len(final_comment) < 120:
            padding = " 请在后续写作中注意审题立意，确保内容紧扣要求。"
            final_comment += padding
            # 修改点: 再次截断到 190
            final_comment = final_comment[:190]
        if len(final_comment) < 10: # Failsafe
             # 修改点: 截断到 190
             final_comment = f"作文《{title}》的切题度评价为{predicted_classification}，建议根据题目要求调整。"[:190]

        logger.info(f"[{essay_id_track1_str}] 使用备用评语 (长度 {len(final_comment)}): {final_comment}")

    end_time = time.time()
    duration = end_time - start_time
    logger.info(f"[{essay_id_track1_str}] 处理完成。耗时: {duration:.2f} 秒.")

    # 更新缓存 (用 Track 2 ID)
    processed_comments_cache[essay_id_track2_str] = final_comment
    save_processed_comments(processed_comments_cache)

    # 返回结果 (用 Track 1 ID)
    return {"id": essay_id_track1, "comment": final_comment}


# --- 主处理流程 (修改长度校验) ---
def process_all_essays(
    essays_data: List[Dict[str, Any]],
    classification_results: Dict[str, str],
    samples_data: List[Dict[str, Any]]
) -> List[Dict[str, Any]]:
    logger.info("开始处理所有作文评论生成...")
    processed_comments_cache = load_processed_comments()
    logger.info(f"从缓存文件 '{processed_comments_file}' 加载 {len(processed_comments_cache)} 条记录。")
    results = []
    essays_to_process = []
    final_results_map = {} # Use map for easier lookup by track1_id
    total_input_essays = len(essays_data)
    logger.info(f"共收到 {total_input_essays} 篇作文数据。")
    processed_count_from_cache = 0

    # --- 筛选并加载缓存 (修改点: 校验长度 120-190) ---
    for essay_data in essays_data:
        essay_id_track1 = essay_data.get("track1_id")
        essay_id_track2 = essay_data.get("id")
        if essay_id_track1 is None or essay_id_track2 is None:
            logger.warning(f"跳过缺 ID 数据: {str(essay_data)[:100]}...")
            continue
        essay_id_track1_str = str(essay_id_track1)
        essay_id_track2_str = str(essay_id_track2)
        if essay_id_track2_str in processed_comments_cache:
            cached_comment = processed_comments_cache[essay_id_track2_str]
            # 修改点: 校验长度 120-190
            if isinstance(cached_comment, str) and 120 <= len(cached_comment) <= 190:
                logger.debug(f"Track1 ID {essay_id_track1_str} 评论从缓存加载 (Key: {essay_id_track2_str})。")
                final_results_map[essay_id_track1] = {"id": essay_id_track1, "comment": cached_comment}
                processed_count_from_cache += 1
            else:
                # 修改点: 更新日志中的长度信息
                logger.warning(f"Track1 ID {essay_id_track1_str} 缓存评论无效 (长度 {len(cached_comment)}，要求 [120, 190])，重新生成。")
                essays_to_process.append(essay_data)
                if essay_id_track2_str in processed_comments_cache:
                    del processed_comments_cache[essay_id_track2_str]
        else:
            logger.debug(f"Track1 ID {essay_id_track1_str} 未在缓存找到 (Key: {essay_id_track2_str})，需处理。")
            essays_to_process.append(essay_data)

    logger.info(f"{processed_count_from_cache} 篇评论从缓存加载。需新生成 {len(essays_to_process)} 篇。")

    # --- 处理新生成 ---
    if essays_to_process:
        progress_bar = tqdm(essays_to_process, desc="生成评论", total=len(essays_to_process))
        newly_processed_count = 0
        for i, essay_data in enumerate(progress_bar):
            essay_id_track1 = essay_data.get("track1_id")
            essay_id_track1_str = str(essay_id_track1)
            current_overall_index = processed_count_from_cache + i + 1
            progress_bar.set_description(f"生成评论 ({current_overall_index}/{total_input_essays}) ID: {essay_id_track1_str}")
            try:
                result = generate_essay_comment(essay_data, classification_results, samples_data, processed_comments_cache)
                if result and isinstance(result, dict) and "id" in result and "comment" in result:
                    track1_id = result["id"]
                    final_comment = result["comment"]
                    # Validate comment again before adding to map (修改点: 校验长度 120-190)
                    if isinstance(final_comment, str) and 120 <= len(final_comment) <= 190:
                        final_results_map[track1_id] = result
                        newly_processed_count += 1
                    elif "ERROR:" in final_comment:
                         logger.error(f"Track1 ID {track1_id} 返回错误: {final_comment}")
                         final_results_map[track1_id] = {"id": track1_id, "comment": "评论生成失败"} # Placeholder
                    else:
                         # 修改点: 更新日志中的长度信息
                        logger.error(f"Track1 ID {track1_id} 最终评论无效(长度 {len(final_comment)}，要求 [120, 190])。")
                        final_results_map[track1_id] = {"id": track1_id, "comment": "评论长度不合规"} # Placeholder
                else:
                    failed_track1_id = essay_data.get("track1_id", "未知")
                    logger.error(f"Track1 ID {failed_track1_id} 返回无效结果: {result}")
                    if failed_track1_id != "未知":
                        final_results_map[failed_track1_id] = {"id": failed_track1_id, "comment": "评论生成函数异常"}
            except Exception as e:
                failed_track1_id = essay_data.get("track1_id", "未知")
                logger.error(f"处理 Track1 ID {failed_track1_id} 出错: {e}", exc_info=True)
                if failed_track1_id != "未知":
                    final_results_map[failed_track1_id] = {"id": failed_track1_id, "comment": "处理过程严重错误"}

            # --- 内存清理 ---
            if (i + 1) % 15 == 0 and i > 0:
                logger.info(f"已处理 {i + 1} 篇新评论，尝试清理内存...")
                try:
                    collected_count = gc.collect()
                    logger.debug(f"gc.collect() 完成，回收 {collected_count} 对象。")
                    if torch.cuda.is_available():
                        torch.cuda.empty_cache()
                        logger.debug("torch.cuda.empty_cache() 完成。")
                    logger.info("内存清理尝试完成。")
                except Exception as mem_e:
                    logger.error(f"内存清理出错: {mem_e}", exc_info=True)
        progress_bar.close()
        logger.info(f"新生成 {newly_processed_count} 篇有效评论。")
    else:
        logger.info("无新评论需生成。")

    # --- 排序和格式化 ---
    logger.info("按 Track 1 ID 顺序格式化最终结果...")
    all_track1_ids = set(data.get("track1_id") for data in essays_data if data.get("track1_id") is not None)
    if classification_results:
        for k in classification_results.keys():
            try:
                all_track1_ids.add(int(k))
            except ValueError:
                logger.warning(f"分类结果含非整数键: {k}")
    if not all_track1_ids:
        logger.error("无有效 Track 1 ID 范围。")
        return []

    max_track1_id = max(all_track1_ids)
    final_output_list = []
    missing_comments = 0
    for track1_id_int in range(max_track1_id + 1):
        if track1_id_int in final_results_map:
            final_output_list.append(final_results_map[track1_id_int])
        else:
            # Add placeholder if an ID within the range is missing entirely
            logger.warning(f"未找到 Track 1 ID {track1_id_int} 结果，用占位符。")
            final_output_list.append({"id": track1_id_int, "comment": "未找到该作文有效评论"})
            missing_comments += 1

    if missing_comments > 0:
         logger.warning(f"最终结果缺 {missing_comments} 条有效评论。")

    logger.info(f"最终格式化结果含 {len(final_output_list)} 条记录 (Track1 ID 0-{max_track1_id})。")
    return final_output_list


# --- 数据加载函数 (保持不变) ---
def load_data(file_path):
    if not os.path.exists(file_path):
        logger.error(f"数据文件未找到: {file_path}")
        return []
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        if not isinstance(data, list):
            logger.error(f"数据文件顶层不是列表: {file_path}")
            return []
        processed_data = []
        seen_track2_ids = set()
        duplicate_track2_id_count = 0
        for i, item in enumerate(data):
            if not isinstance(item, dict):
                logger.warning(f"跳过索引 {i} 非字典项: {type(item)}")
                continue
            # Add track1_id (index)
            item["track1_id"] = i
            # Check track2_id (original id)
            track2_id = item.get("id")
            if track2_id is None:
                logger.warning(f"索引 {i} (Track1 ID: {i}) 缺原始 Track 2 ID。跳过此条目。")
                continue # Strictly require track2_id for caching
            track2_id_str = str(track2_id)
            if track2_id_str in seen_track2_ids:
                logger.warning(f"重复 Track 2 ID '{track2_id_str}' 在索引 {i} (Track1 ID: {i})，跳过。")
                duplicate_track2_id_count += 1
                continue
            seen_track2_ids.add(track2_id_str)
            processed_data.append(item)
        if duplicate_track2_id_count > 0:
            logger.warning(f"因 Track 2 ID 重复跳过 {duplicate_track2_id_count} 条记录。")
        logger.info(f"成功加载并处理 {len(processed_data)} 条有效数据，已添加 Track 1 ID。")
        return processed_data
    except json.JSONDecodeError as e:
        logger.error(f"解析数据文件失败: {file_path} - {e}")
        return []
    except Exception as e:
        logger.error(f"加载数据出错: {e}", exc_info=True)
        return []

# --- 保存结果函数 (保持不变) ---
def save_results(results: List[Dict[str, Any]], output_path: str):
    try:
        output_dir = os.path.dirname(output_path)
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir)
            logger.info(f"创建目录: {output_dir}")
        temp_file_path = output_path + ".tmp"
        with open(temp_file_path, 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)
        os.replace(temp_file_path, output_path)
        logger.info(f"结果成功保存到 {output_path}")
    except Exception as e:
        logger.error(f"保存结果到 {output_path} 失败: {e}", exc_info=True)
        if os.path.exists(temp_file_path):
            try:
                os.remove(temp_file_path)
            except OSError:
                pass

# --- 主函数 (保持不变) ---
def main():
    overall_start_time = time.time()
    logger.info("="*30 + " 开始评论生成任务 (Track 2) " + "="*30)
    logger.info(f"加载输入数据: {input_file}")
    data = load_data(input_file)
    if not data:
        logger.error("加载输入数据失败，终止。")
        return
    logger.info(f"加载 Track 1 分类结果: {results_file}")
    classification_results = load_classification_results(results_file)
    if not classification_results:
        logger.warning("未加载分类结果，将使用默认 '一般'。")
    logger.info(f"加载样本数据: {samples_file}")
    samples_data = load_samples(samples_file)
    if not samples_data:
        logger.warning("未加载有效样本数据。")
    logger.info("开始处理所有作文...")
    final_results = process_all_essays(data, classification_results, samples_data)
    if final_results:
        # Validate final count against expected range
        expected_count = 0
        all_track1_ids = set(d.get("track1_id") for d in data if d.get("track1_id") is not None)
        if classification_results:
             for k in classification_results.keys():
                  try:
                      all_track1_ids.add(int(k))
                  except ValueError:
                      pass
        if all_track1_ids:
             expected_count = max(all_track1_ids) + 1

        if expected_count > 0 and len(final_results) != expected_count:
            logger.warning(f"最终结果数 ({len(final_results)}) 与预期 ({expected_count}) 不符。")
        elif expected_count == 0:
            logger.warning("无法确定预期结果数。")

        logger.info(f"准备保存 {len(final_results)} 条评论到 {output_file}")
        save_results(final_results, output_file)
    else:
        logger.warning("无有效评论结果生成，不保存文件。")
    overall_end_time = time.time()
    total_duration = overall_end_time - overall_start_time
    processed_count = len(final_results) if final_results else 0
    logger.info(f"评论生成处理完成。共处理 {processed_count} 条记录。")
    logger.info(f"总耗时: {total_duration:.2f} 秒 ({total_duration/60:.2f} 分钟)。")
    if processed_count > 0 and total_duration > 0.1:
        avg_time = total_duration / processed_count
        logger.info(f"平均每条耗时: {avg_time:.2f} 秒。")
    logger.info("="*30 + " 评论生成任务结束 " + "="*30)

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        logger.info("程序被用户中断。")
    except Exception as e:
        logger.critical(f"主程序严重错误: {e}", exc_info=True)